In [1]:
import pandas as pd
import numpy as np
import math
import os

# import for time, and file change
import json
from PIL import Image

# frequent updating
import time
import random

In [2]:
# define filepaths
workspace = "/Users/advaysingh/Documents/projects/hindi_classification/" 
#print("Current workspace:", workspace)

data = os.path.join(workspace, 'data/Hindi/')
dict_lib = os.path.join(workspace, 'data/dict.csv')
img_path = os.path.join(workspace, 'server/snapshot.png')
out_file = os.path.join(workspace, 'server/outputs.json')


In [3]:
# create train/test dicts with files and labels

def create_dict(x: str) -> dict:
    x_dict = {}
    i = 0
    for dir in os.listdir(os.path.join(data, x)):
        for file in os.listdir(os.path.join(data, x, dir)):
            x_dict[os.path.join(data, x, dir, file)] = i
        i += 1
    return x_dict

# make pandas df

train_df = pd.DataFrame.from_dict(create_dict('Train'), orient='index')
test_df = pd.DataFrame.from_dict(create_dict('Test'), orient='index')

train_df.head()

,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/26446.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/62562.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/74300.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/86182.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/62576.png,0


In [224]:
# Activations class
class Activation:
    # can add more types
    def __init__(self, act_type: str) -> None:
        self.act = act_type

    def print_act(self) -> None:
        return self.act
    
    def compute(self, z: float) -> float:

        # Sigmoid function
        if (self.act == 'sigmoid'):
            return 1 / (1 + math.exp(z))
        
        # ReLU function
        elif (self.act == 'relu'):
            if (z < 0): return 0
            return z

    def prime(self, z: float) -> float:

        # Sigmoid prime
        if (self.act == 'sigmoid'):
            return (self.compute(z)) * (1 - self.compute(z))
        
        # ReLU prime
        if (self.act == 'relu'):
            if (z < 0): return 0
            return 1


def softmax(i: int, costs: list) -> float:
    return costs[i] / sum(costs)

def softmax_prime(i: int, costs: list) -> float:
    return softmax(i, costs) * (1 - softmax(i, costs))


    
a = Activation('sigmoid')

print(a.compute(233.2))

5.278698047466735e-102


In [249]:
class Model:
    def __init__(self, train_df: pd.DataFrame,
                  act: Activation, layers: list, step: float, stochastic: bool) -> None:
        self.train_df = train_df

        # create activation function
        self.act = act
        self.layers = layers

        # init weight and bias with # layers np arrays
        self.all_weights = [None] * (int(len(layers)))
        self.all_bias = [None] * (int(len(layers)))
        for i in range(len(layers)):
            self.all_weights[i] = self.random_arrs(i)
            self.all_bias[i] = self.random_arrs(i)
        for val in self.all_weights:
            print(val.shape)

        # init learn speed
        self.speed = step

        # determine gradient decent type
        self.stochastic = stochastic

    def random_arrs(self, layer: int):
        if (layer == 0):
            return(np.random.rand(self.layers[layer], 1024))
        else:
            return(np.random.rand(self.layers[layer], self.layers[layer - 1]))

    def img_to_np(self, dir) -> np.array:
        return np.array(Image.open(dir)).flatten()
    
    def print_img(self) -> None:
        for img_dir, row in self.train_df.iterrows():
            print(self.img_to_np(img_dir), row[0])

    def print_weights(self) -> None:
        for arr in self.all_weights:
            print(arr.shape)

    def prop_forward(self, inputs: list, layer: int) -> list:
        a_vals = [0.0] * int(self.layers[layer + 1])
        for a in range(len(a_vals)):
            for x in range(len(inputs) - 1):
                if (layer == int(len(self.layers) - 2)):
                    #TODO: determine .product formula
                    a_vals[a] += softmax(x, inputs)
                else: 
                    a_vals[a] += self.act.compute(self.all_weights[layer][a, x] * inputs[x]+ self.all_bias[layer][a, x])
        return a_vals
    


    def prop_backword(self, x: list, label: int, layer: int):
        outputs = self.prop_forward(x, layer)
        if (layer == len(self.layers) - 2):
            y = self.vals_for_softmax(label)
            for i in range(len(y) - 1):
                for j in range(len(x) - 1):
                    val = (y[i] - outputs[i]) * softmax_prime(i, outputs) 
                    self.all_weights[layer + 1][i][j] += val* x[j]
                    self.all_bias[layer + 1][i][j] += val
            return(val)

        for output in range(len(outputs)):
            for input in range(len(x) - 1):
                val = 0.0
                val += float(self.prop_backword(outputs, label, layer + 1)) * float(sum(self.all_weights[layer + 1][output])) + self.act.prime(x[input]) #edit
                print(val)
                self.all_weights[layer][output][input] += val * x[input]
                self.all_bias[layer][output][input] += val
        return val

            
        # recurse to the following layer
        #else:
            #self.all_weights[layer][]

        # prop_forward for all of the layers
        print("in progress")
        # change weights appropriatly
    # potentially reconsider for time complexity
    def vals_for_softmax(self, index: int) -> list:
        vals = self.train_df[0].unique()
        for i in range(len(vals)):
            if vals[i] != index:
                vals[i] = 0
            else:
                vals[i] = 1
        return vals


    def train(self, epochs: int) -> None:
        # call back prop with dataset epochs times
        for epoch in  range(epochs):
            for img_dir, row in self.train_df.iterrows():
                self.prop_backword(self.img_to_np(img_dir).tolist(), row[0], 0)
                print(epoch)

    def classify(self, img: np.array) -> int:
        print("classification in progress")

    def test(self, test_df: pd.DataFrame) -> None:
        for img_dir, row in test_df.iterrows():
            print(self.img_to_np(img_dir), row[0])

    def save(self) -> None:
        print("in_progress") # save into csv file. Later use this file to prop forwars predicting the outcome

    def print_weights(self, layer: int) -> None:
        for row in self.all_weights[layer]:
            for weight in row:
                print(weight)



In [250]:
cnn = Model(train_df, Activation('sigmoid'), [10, 10, 10, 10, 46], 1.2, False)
cnn.train(3)

(10, 1024)
(10, 10)
(10, 10)
(10, 10)
(46, 10)
46
46
0.1478515637036461
46
46
0.1478515637241959
46
46
0.14785156372419545
46
46
0.14562872897806328
46
46
0.1478515637241959
46
46
0.1478515637226911
46
46
0.1478515637241959
46
46
0.1478515637241959
46
46
0.14785120490311682
46
46
0.19748643499941723
46
46
0.197486435019967
46
46
0.19748643501996654
46
46
0.19526360027383438
46
46
0.197486435019967
46
46
0.1974864350184622
46
46
0.197486435019967
46
46
0.197486435019967
46
46
0.19748607619888792
46
46
0.16332484012439252
46
46
0.1633248401449423
46
46
0.16332484014494186
46
46
0.1611020053988097
46
46
0.1633248401449423
46
46
0.1633248401434375
46
46
0.1633248401449423
46
46
0.1633248401449423
46
46
0.16332448132386324
46
46
0.17728005432550345
46
46
0.1772800543460532
46
46
0.17728005434605276
46
46
0.1750572195999206
46
46
0.1772800543460532
46
46
0.1772800543445484
46
46
0.1772800543460532
46
46
0.1772800543460532
46
46
0.17727969552497413
46
46
0.1337318369835897
46
46
0.13373183700

OverflowError: math range error

In [139]:
from numpy.random import randint
cnn = Model(train_df, Activation('sigmoid'), [10, 10, 10, 46], 1.2, False)
#cnn.print_img()
#cnn.print_weights()
some_vals = cnn.prop_forward([1, 2, 3, 45, 5], 3)
#cnn.print_weights(3)
#print(cnn.vals_for_softmax(3))
#print (some_vals)
#cnn.test(test_df)
cnn.prop_backward(randint(0, 20, 46), cnn.vals_for_softmax(10), 3)

[-0.01767677 -0.01262626 -0.01262626 -0.01010101 -0.03282828 -0.00252525
 -0.04545455 -0.03030303 -0.0479798   0.          0.97222222 -0.01767677
 -0.01767677 -0.02272727 -0.02777778 -0.00505051 -0.04040404 -0.01767677
  0.         -0.03030303 -0.02525253 -0.02777778 -0.00252525 -0.02525253
 -0.00505051 -0.01010101 -0.04040404 -0.04292929 -0.03282828 -0.00757576
 -0.03787879 -0.03787879 -0.00757576 -0.02525253 -0.02525253 -0.01010101
 -0.02272727 -0.00252525 -0.00252525 -0.02777778 -0.03535354 -0.04292929
 -0.04292929 -0.02272727  0.         -0.01767677]
in progress


In [102]:
cnn.prop_backward([1, 2, 3, 45, 5], some_vals, [-10] * 46, 3)
cnn.print_weights(3)

in progress
1.6461973679090411
2.445717803365009
2.9716765614213325
36.84718333932656
0.6948725533065305
0.11151094740007084
0.6154698626025906
0.4486772550252789
0.2876354902712329
0.10971120961198844
1.6679855428588126
2.856368095225691
3.4320935250418843
49.50315407127205
0.6975998099079385
0.35140079867870166
0.08063390681210902
0.8979808184715279
0.7343854103587517
0.4267909741611481
3.014712831115513
4.482171045478331
6.8167337857046135
92.85552627823884
0.6156446417294256
0.612368698763382
0.62883147417825
0.9223552003491825
0.3261563088073026
0.5482358236845165
11.776330109598462
23.504581768631507
35.93978578905343
526.75271326049
0.042424517069591494
0.6291086622614546
0.3182744228101819
0.710877398732458
0.9148925166019155
0.10874452376579713
1.943094440034948
3.6328145733821815
5.897872327214269
75.17055315672786
0.5405930815239788
0.568328911493736
0.8482307401028797
0.62651314320018
0.3005401299455076
0.0032119207939608208
3.273746130932874
6.609184617412231
8.47614669434

In [109]:
import json
import time
import random

from PIL import Image


def update_json_file(hindi_value, english_value):
    with open(out_file, 'r') as file:
        data = json.load(file)
    data['hindi'] = hindi_value
    data['english'] = english_value
    with open(out_file, 'w') as file:
        json.dump(data, file, indent=4)

# Define a list of possible values if applicable
hindi_values = ["हिं", "नया", "मूल्य", "उदाहरण"]
english_values = ["Eng", "New", "Value", "Example"]

# Loop to update the JSON file every second with a random value
try:
    while True:
        # Generate a random value from the lists
        new_hindi_value = random.choice(hindi_values)
        new_english_value = random.choice(english_values)

        update_json_file(new_hindi_value, new_english_value)
        
        # Print the new values (for debug purposes)
        # print(f'Updated JSON file with hindi: {new_hindi_value}, english: {new_english_value}')

        # read in and resize img
        img = Image.open(img_path).convert("L").resize((32, 32))
 
        # read in image for debugging
        """img.show("img to classify")
        print(img.format)
        print(img.mode)
        print(img.size)"""

        # create np array and resize
        img_data = np.array(img).flatten()
        print(img_data.shape)
        print(img_data)

        
        time.sleep(10)  # Wait for 1 second
except KeyboardInterrupt:
    print("Update stopped by user.")

(1024,)
[0 0 0 ... 0 0 0]
Update stopped by user.
